<h1>Getting engagement data for tweets Twitter Climate Change sentiment dataset</h1>

In [1]:
import pandas as pd
import numpy as np
import tweepy 
import os

<h2>Load Data</h2>

In [2]:
tweets = pd.read_csv("../twitter_sentiment_data.csv")

In [3]:
tweets.head()

,sentiment,message,tweetid
0,-1,@tiniebeany climate change is an interesting h...,792927353886371840
1,1,RT @NatGeoChannel: Watch #BeforeTheFlood right...,793124211518832641
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256
3,1,RT @Mick_Fanning: Just watched this amazing do...,793124635873275904
4,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125156185137153


<h2>Extract Engagement</h2>

In [4]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAOK%2BdQEAAAAAnDXO25BoSCGC7MIIrXvgHR%2BU84c%3DOfoTzDxeli0xvZM9JwmnTNjxv8sCoilcpmJ4YvuFy243iq6zvt"
client = tweepy.Client(bearer_token)

In [5]:
def extract_entities(t):
    keys = list(t.data.entities.keys())
    if 'mentions' in keys:
        mentioned_users = [i['username'] for i in t.data.entities['mentions']]
    else:
        mentioned_users = None
    if 'hashtags' in keys:
        hashtags = [i['tag'] for i in t.data.entities['hashtags']]
    else: 
        hashtags = None
    if 'annnotations' in keys:
        annotations = [i['annotation'] for i in t.data.entities['annotations']]
    else: 
        annotations = None
    if 'urls' in keys:
        urls = [i['url'] for i in t.data.entities['urls']]
    else: 
        urls = None
    return (mentioned_users, hashtags, annotations, None)
        

In [26]:
def get_tweet_data(t):
    t = client.get_tweet(t, tweet_fields=['entities', 'public_metrics', 'author_id'])
    if t.data == None:
        return []
    #identifiers
    tweet_id = t.data.id
    tweeter_id = t.data.author_id
    #get tweet attributes
    if getattr(t.data, 'public_metrics') != None:
        metrics = t.data.public_metrics
        retweets = metrics['retweet_count']
        replies = metrics['reply_count']
        likes = metrics['like_count']
        quote_tweets = metrics['quote_count']
    else:
        retweets = None
        replies = None
        likes = None
        quote_tweets = None
    #get tweet entities
    if getattr(t.data, 'entities'):
        mentioned_users, hashtags, annotations, urls = extract_entities(t)
    else:
        mentioned_users, hashtags, annotations, urls = None, None, None, None 
    #write to file
    data = np.array([tweet_id, tweeter_id, retweets, replies, likes, quote_tweets, mentioned_users,
                     hashtags, annotations, urls], dtype=object)
    data_df = pd.DataFrame([data], columns = ['tweet_id', 'tweeter_id', 'retweets', 'replies', 'likes', 
                                              'quote_tweets', 'mentioned_users', 'hashtags', 'annotations', 
                                              'urls'])
    data_df.to_csv("tweet_engagement.csv", mode="a", index=False, header=True)
    return None

In [27]:
ids = tweets['tweetid'].values[:100]
engagement_data = [get_tweet_data(i) for i in ids]

['annotations', 'mentions', 'urls']
['urls']
['urls', 'annotations', 'mentions', 'hashtags']
['annotations', 'hashtags', 'urls', 'mentions']
['urls', 'annotations']
['hashtags', 'annotations', 'mentions', 'urls']
['urls', 'mentions']
['urls']
['annotations', 'urls']
['urls', 'annotations', 'hashtags']
['urls', 'annotations', 'mentions', 'hashtags']
['urls', 'mentions', 'hashtags']
['mentions', 'urls', 'annotations', 'hashtags']
['urls', 'hashtags', 'annotations', 'mentions']
['annotations', 'urls']
['urls', 'hashtags', 'annotations']
['hashtags', 'urls', 'annotations']
['annotations', 'urls', 'hashtags']
['mentions', 'urls']


In [28]:
test = pd.read_csv("tweet_engagement.csv")

In [30]:
test

,tweet_id,tweeter_id,retweets,replies,likes,quote_tweets,mentioned_users,hashtags,annotations,urls
0,792927353886371840,2986652667,0,0,0,0,['tiniebeany'],NaN,NaN,NaN
1,tweet_id,tweeter_id,retweets,replies,likes,quote_tweets,mentioned_users,hashtags,annotations,urls
2,793124635873275904,257517997,21,0,0,0,['Mick_Fanning'],NaN,NaN,NaN
3,tweet_id,tweeter_id,retweets,replies,likes,quote_tweets,mentioned_users,hashtags,annotations,urls
4,793125156185137153,555506562,10,0,0,0,['cnalive'],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
80,793168974510821376,62050691,0,0,0,0,NaN,"['weather', 'forecast']",NaN,NaN
81,tweet_id,tweeter_id,retweets,replies,likes,quote_tweets,mentioned_users,hashtags,annotations,urls
82,793169535264096256,366888493,3,0,0,0,['CatalyticRxn'],['sciwri16'],NaN,NaN
83,tweet_id,tweeter_id,retweets,replies,likes,quote_tweets,mentioned_users,hashtags,annotations,urls


In [19]:
engagement_data[0]

array([792927353886371840, 2986652667, 0, 0, 0, 0, list(['tiniebeany']),
       None, None, None], dtype=object)

In [23]:
pd.DataFrame(data = [engagement_data[0]], columns = ['tweet_id', 'tweeter_id', 'retweets', 'replies', 
                                                     'likes', 'quote_tweets', 'mentioned_users', 
                                                     'hashtags', 'annotations', 'urls']
            )

,tweet_id,tweeter_id,retweets,replies,likes,quote_tweets,mentioned_users,hashtags,annotations,urls
0,792927353886371840,2986652667,0,0,0,0,[tiniebeany],None,None,None


In [ ]:
t = client.get_tweet(1576673769644359680, tweet_fields=['entities', 'public_metrics', 'author_id'], 
          user_fields = ['description'], expansions=['author_id'])

In [ ]:
temp = pd.DataFrame([1576673769644359680])
# .apply(get_tweet_data, axis=1)

In [ ]:
t.data.entities

In [ ]:
dir(t.data)

In [ ]:
dir(t.includes)

<h2>Write to file</h2>

In [ ]:
df = pd.DataFrame(data=engagement_data, columns = ['tweet_id', 'tweeter_id', 'retweets', 
                                                   'replies', 'likes', 'quote_tweets', 
                                                   'mentioned_users', 'hashtags', 'annotations'])

In [ ]:
os.chdir("../")

In [ ]:
df.to_csv("engagement_data.csv")